<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports-and-set-up" data-toc-modified-id="Imports-and-set-up-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports and set up</a></span></li><li><span><a href="#Build-a-better-dictionary-dataset" data-toc-modified-id="Build-a-better-dictionary-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Build a better dictionary dataset</a></span><ul class="toc-item"><li><span><a href="#Partition-by-Letter" data-toc-modified-id="Partition-by-Letter-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Partition by Letter</a></span></li><li><span><a href="#Convert-to-Trees" data-toc-modified-id="Convert-to-Trees-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Convert to Trees</a></span></li></ul></li><li><span><a href="#Search-for-words" data-toc-modified-id="Search-for-words-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Search for words</a></span></li><li><span><a href="#Identify-pangram" data-toc-modified-id="Identify-pangram-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Identify pangram</a></span></li></ul></div>

## Imports and set up

In [2]:
import dill
import os
from collections import namedtuple


alphabet = list('abcdefghijklmnopqrstuvwxyz')
parameters = ['parent', 'letter', 'wordToHere', 'terminatingWord']+alphabet
defaults = [None, '', '', False]+[None for _ in alphabet]
LetterNode = namedtuple('LetterNode', parameters, defaults=defaults)    

# root = LetterNode()
# root = root._replace(a='test')

## Build a better dictionary dataset

See [Data/README.txt](./Data/README.txt) for info on the dictionary dataset. I downloaded words_alpha.txt because it cuts out words that have numbers and special symbols.



### Partition by Letter

First further cut this down to words with 4 or more letters since we need those for Spelling Bee. Then save them to files by starting letter. 

In [69]:
# Delete letter files
# letterFiles = os.listdir('./Data/LetterFiles')
# for f in letterFiles:
#   os.remove('./Data/LetterFiles/'+f)

In [70]:
with open('./Data/words_alpha.txt', 'r') as file:
  words = file.readlines()

# Make separate files by letter
# I commented this out so that it won't accidentally be rerun
# for word in words:
#   if len(word.strip()) < 4:
#     continue
#   word = word.lower()
#   startingLetter = word[0]
#   with open(f'./Data/LetterFiles/{startingLetter}.txt', 'a+') as letterFile:
#     letterFile.write(word)

### Convert to Trees

In [3]:
# Delete current trees
# letterTrees = os.listdir('./Data/LetterTrees')
# for f in letterTrees:
#   os.remove('./Data/LetterTrees/'+f)

In [3]:
def returnRoot(node):
  """
  Not using this right now but it returns you to the top of a tree.
  
  """
  while node.parent is not None:
    node = node.parent
  return node

def updateNode(node, word):
  """
  This is a recursive function that builds out the tree for a given word. 
  """
  if len(word) == 0:
    return None
  
  assert node.letter == word[0]
  
  if len(word) == 1:
    # mark terminatingWord True because word ends here
    node = node._replace(terminatingWord=True)
  else:
    nextLetter = word[1]
    currentNextNode = node.__getattribute__(nextLetter)
    if currentNextNode is None:
      # make node if it doesn't exist
      currentNextNode = LetterNode(parent=node, letter=nextLetter, wordToHere=node.wordToHere+nextLetter)
    newNextNode = updateNode(currentNextNode, word[1:])
    node = node._replace(**{nextLetter:newNextNode})
  
  return node
   

In [4]:
# test building tree for one letter
letter = 'i'
with open(f'./Data/LetterFiles/{letter}.txt', 'r+') as letterFile:
  lettersWords = letterFile.readlines()
  
root = LetterNode(wordToHere=letter, letter=letter)
for word in lettersWords:
  word = word.strip()
  root = updateNode(root, word)


In [5]:
# build trees for each letter
alphabet = list('abcdefghijklmnopqrstuvwxyz')
for letter in alphabet:
  # read words for letter
  with open(f'./Data/LetterFiles/{letter}.txt', 'r+') as letterFile:
    lettersWords = letterFile.readlines()

  # build tree
  root = LetterNode(wordToHere=letter, letter=letter)
  for word in lettersWords:
    word = word.strip()
    root = updateNode(root, word)

  # pickle tree
  with open(f'./Data/LetterTrees/{letter}.tree', 'wb') as treeFile:
    dill.dump(root, treeFile)
    
  print(f"Finished letter {letter}")

Finished letter a
Finished letter b
Finished letter c
Finished letter d
Finished letter e
Finished letter f
Finished letter g
Finished letter h
Finished letter i
Finished letter j
Finished letter k
Finished letter l
Finished letter m
Finished letter n
Finished letter o
Finished letter p
Finished letter q
Finished letter r
Finished letter s
Finished letter t
Finished letter u
Finished letter v
Finished letter w
Finished letter x
Finished letter y
Finished letter z


## Search for words

In [6]:
def searchTree(node, puzzle, foundWords=[], mandatoryLetter=None):
  if node.terminatingWord:
    wordToHere = node.wordToHere
    if mandatoryLetter is not None:
      if mandatoryLetter in wordToHere:
#         print(node.wordToHere)
        foundWords.append(node.wordToHere)
    else:
      foundWords.append(node.wordToHere)
  for l in puzzle:
    child = node.__getattribute__(l)
    if child is not None:
      foundWords = searchTree(child, puzzle, foundWords, mandatoryLetter)
  return foundWords

# example = 'ltvpexi'
# foundWords = searchTree(root, example, mandatoryLetter='i')
# foundWords

In [7]:
example = 'iltvpex'
mandatoryLetter = example[0]

allFoundWords = []
for letter in example:
  with open(f'./Data/LetterTrees/{letter}.tree', 'rb') as treeFile:
    thisRoot = dill.load(treeFile)
  foundWords = searchTree(node=thisRoot, puzzle=example, foundWords=[], mandatoryLetter=mandatoryLetter)
  allFoundWords.extend(foundWords)
  

In [8]:
print(len(allFoundWords))
print(len(set(allFoundWords)))
allFoundWords

113
113


['illite',
 'illipe',
 'ilex',
 'itll',
 'itel',
 'ipil',
 'ipilipil',
 'ieee',
 'ixil',
 'ixtle',
 'lill',
 'lilt',
 'lile',
 'liti',
 'little',
 'lite',
 'live',
 'liplet',
 'lippie',
 'lieve',
 'lixive',
 'levi',
 'levite',
 'leptite',
 'leepit',
 'till',
 'tillite',
 'tillet',
 'tilt',
 'tile',
 'tilette',
 'titi',
 'titivil',
 'title',
 'tittie',
 'tittle',
 'tite',
 'tipi',
 'tipit',
 'tipiti',
 'tiple',
 'tiplet',
 'tiptilt',
 'tipple',
 'tippet',
 'tippee',
 'tipe',
 'teil',
 'teli',
 'telei',
 'tettix',
 'textile',
 'viii',
 'vili',
 'vill',
 'villi',
 'ville',
 'vile',
 'viti',
 'vittle',
 'vite',
 'vive',
 'vielle',
 'vlei',
 'veil',
 'vetitive',
 'vexil',
 'pili',
 'pill',
 'pillet',
 'pile',
 'pilei',
 'pittite',
 'pitpit',
 'pipi',
 'pipil',
 'pipile',
 'pipit',
 'pipple',
 'pipe',
 'pipet',
 'pipette',
 'pielet',
 'piet',
 'piete',
 'pixie',
 'pixel',
 'plie',
 'pelite',
 'pellile',
 'petit',
 'petite',
 'petti',
 'peelite',
 'elite',
 'elix',
 'evil',
 'evite',
 'epil',

## Identify pangram

In [23]:
findPangrams(example, allFoundWords)

expletive
